# Lab 03 — Interoperabilidad: De SQL a FHIR

**BE3006 · Análisis de Datos Biomédicos · Ciclo 1, 2026**

---

En **Lab 1** construiste un EHR relacional.  
En **Lab 2** normalizaste su semántica con ICD-10 y LOINC.

Ahora el paso final: convertir esos datos a **FHIR** (Fast Healthcare Interoperability Resources),
el estándar HL7 que usan hospitales, gobiernos y aplicaciones de salud para intercambiar
información clínica.

---

```
┌──────────────┐     ┌─────────────────────┐    ┌─────────────────┐
│   EHR (SQL)  │───▶│  Terminología       │───▶│  FHIR Bundle    │
│  patients    │     │  ICD-10 / LOINC     │    │  Patient        │
│  admissions  │     │  mapeos             │    │  Encounter      │
│  diagnoses   │     └─────────────────────┘    │  Condition      │
│  labevents   │                                │  Observation    │
└──────────────┘                                └─────────────────┘
```

---

**Objetivos:**

1. Construir el EHR y el servidor de terminología desde el notebook (sin correr scripts manualmente)
2. Ver cómo cada tabla SQL se transforma en un recurso FHIR
3. Entender cómo los recursos FHIR se referencian entre sí
4. Ensamblar un Bundle FHIR listo para intercambio
5. Consultar la especificación oficial de HL7 FHIR


---

## Parte 0 — Setup y conexión

Asegúrate de que Docker esté corriendo: `docker compose up -d`  
Jupyter se conecta a PostgreSQL usando el hostname `db` (nombre del servicio en Docker Compose).


In [8]:
import json
from pathlib import Path

import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text

from fhir.resources.bundle import Bundle
from fhir.resources.condition import Condition
from fhir.resources.encounter import Encounter
from fhir.resources.observation import Observation
from fhir.resources.patient import Patient

# Conexión con encoding correcto (evita que nombres con tildes aparezcan rotos)
DB_URL = 'postgresql://uvg_user:uvg_password@db:5432/health_data?client_encoding=utf8'
engine = create_engine(DB_URL)

with engine.connect() as conn:
    version = conn.execute(text('SELECT version()')).scalar()

print('Conexión exitosa.')
print(version)

Conexión exitosa.
PostgreSQL 16.11 (Debian 16.11-1.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit


### Función `ejecutar_sql()`

En lugar de pedirte que corras scripts manualmente, este notebook los ejecuta directamente.  
Después de cada script, vemos el resultado en tablas para entender qué cambió.


In [9]:
SQL_DIR = Path('../sql')


def ejecutar_sql(filename, verbose=True):
    """Lee y ejecuta un archivo .sql completo contra la base de datos."""
    sql = (SQL_DIR / filename).read_text(encoding='utf-8')
    conn = psycopg2.connect(
        host='db', dbname='health_data',
        user='uvg_user', password='uvg_password',
        options='-c client_encoding=UTF8'
    )
    try:
        with conn.cursor() as cur:
            cur.execute(sql)  # psycopg2 acepta múltiples statements
        conn.commit()
    finally:
        conn.close()
    if verbose:
        print(f'✓ Ejecutado: {filename}')

### Reset de la base de datos (opcional)

**Primera vez:** no necesitas esto.  
**Si ves datos duplicados o errores al re-ejecutar:** cambia `HACER_RESET = True` y ejecuta esta celda primero.


In [10]:
HACER_RESET = False  # Cambia a True SOLO si necesitas limpiar la DB

if HACER_RESET:
    _reset = '''
        DROP VIEW IF EXISTS v_diagnoses_coded CASCADE;
        DROP VIEW IF EXISTS v_labitems_coded CASCADE;
        DROP TABLE IF EXISTS labevents CASCADE;
        DROP TABLE IF EXISTS d_labitems CASCADE;
        DROP TABLE IF EXISTS diagnoses CASCADE;
        DROP TABLE IF EXISTS admissions CASCADE;
        DROP TABLE IF EXISTS patients CASCADE;
        DROP TABLE IF EXISTS map_labitem_to_concept CASCADE;
        DROP TABLE IF EXISTS map_diagnosis_text_to_concept CASCADE;
        DROP TABLE IF EXISTS terminology_concepts CASCADE;
        DROP TABLE IF EXISTS terminology_systems CASCADE;
    '''
    _conn = psycopg2.connect(
        host='db', dbname='health_data',
        user='uvg_user', password='uvg_password',
        options='-c client_encoding=UTF8'
    )
    try:
        with _conn.cursor() as cur:
            cur.execute(_reset)
        _conn.commit()
    finally:
        _conn.close()
    print('Base de datos limpia. Continúa con la Parte 1.')
else:
    print('Reset omitido.')

Reset omitido.


---

## Parte 1 — El modelo EHR

Antes de hablar de FHIR, necesitamos datos clínicos.  
Vamos a construir el modelo relacional del Lab 1, directamente desde este notebook.

El modelo tiene 5 tablas:

```
patients
│  (subject_id PK, external_id, full_name, sex, date_of_birth)
│
└── admissions  (hadm_id PK, subject_id FK, admittime, dischtime, admission_type)
        │
        ├── diagnoses    (diagnosis_id PK, hadm_id FK, diagnosis_text)
        │
        └── labevents    (labevent_id PK, hadm_id FK, labitem_id FK, charttime, value_num)
                │
                └── d_labitems   (labitem_id PK, label, unit)
```

Las **admisiones** son el eje del modelo: diagnósticos y labs cuelgan de ellas,
no directamente del paciente. Esto refleja la realidad clínica: un laboratorio
pertenece a un _encuentro_, no al paciente en abstracto.


In [36]:
# Crear tablas EHR e insertar datos de muestra
ejecutar_sql('002_ehr_schema.sql')

UniqueViolation: duplicate key value violates unique constraint "patients_external_id_key"
DETAIL:  Key (external_id)=(MRN-0001) already exists.


In [38]:
print('Tablas EHR creadas:')
display(pd.read_sql('''
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_name IN ('patients', 'admissions', 'diagnoses', 'd_labitems', 'labevents')
    ORDER BY table_name;
''', engine))

Tablas EHR creadas:


,table_name
0,admissions
1,d_labitems
2,diagnoses
3,labevents
4,patients


In [39]:
print('=== Pacientes ===')
display(pd.read_sql('SELECT * FROM patients ORDER BY subject_id;', engine))

=== Pacientes ===


,subject_id,external_id,full_name,sex,date_of_birth,date_of_death
0,1,MRN-0001,Ana Lopez,F,1980-03-12,None
1,2,MRN-0002,Carlos Perez,M,1975-07-01,None
2,3,MRN-0003,María Gomez,F,1992-11-23,None
3,4,MRN-0004,Jose Martinez,M,1968-05-09,None
4,5,MRN-0005,Alex Rivera,O,2001-08-14,None
5,6,MRN-0006,Lucía Herrera,F,1988-02-02,None
6,7,MRN-0007,Miguel Castillo,M,1959-10-30,None
7,8,MRN-0008,Sofia Morales,F,1979-06-18,None


In [40]:
print('=== Admisiones ===')
display(pd.read_sql('''
    SELECT
        a.hadm_id,
        p.full_name          AS paciente,
        a.admittime::date    AS ingreso,
        a.dischtime::date    AS egreso,
        a.admission_type     AS tipo,
        a.hospital_expire_flag AS fallecio
    FROM admissions a
    JOIN patients p ON p.subject_id = a.subject_id
    ORDER BY a.hadm_id;
''', engine))

=== Admisiones ===


,hadm_id,paciente,ingreso,egreso,tipo,fallecio
0,1,Ana Lopez,2101-01-10,2101-01-15,Emergency,False
1,2,Ana Lopez,2102-06-01,2102-06-05,Elective,False
2,3,Carlos Perez,2101-03-20,2101-03-28,Emergency,True
3,4,María Gomez,2101-07-11,2101-07-14,Urgent,False
4,5,Jose Martinez,2101-09-02,2101-09-10,Emergency,False
5,6,Alex Rivera,2101-12-18,2101-12-22,Emergency,False
6,7,Lucía Herrera,2101-02-05,2101-02-08,Emergency,False
7,8,Lucía Herrera,2101-11-01,2101-11-04,Urgent,False
8,9,Miguel Castillo,2101-05-14,2101-05-20,Emergency,False
9,10,Sofia Morales,2101-08-21,2101-08-24,Emergency,False


In [41]:
print('=== Diagnósticos (EHR puro) ===')
print('Por ahora la tabla solo tiene diagnosis_text — sin columnas de código.')
print('Las columnas code_system, code, code_display se agregan cuando corramos el script 003.')
display(pd.read_sql('''
    SELECT d.diagnosis_id, d.hadm_id, p.full_name AS paciente,
           d.diagnosis_text
    FROM diagnoses d
    JOIN admissions a ON a.hadm_id = d.hadm_id
    JOIN patients p ON p.subject_id = a.subject_id
    ORDER BY d.diagnosis_id;
''', engine))

=== Diagnósticos (EHR puro) ===
Por ahora la tabla solo tiene diagnosis_text — sin columnas de código.
Las columnas code_system, code, code_display se agregan cuando corramos el script 003.


,diagnosis_id,hadm_id,paciente,diagnosis_text
0,1,1,Ana Lopez,Hypertension
1,2,1,Ana Lopez,Chest pain
2,3,2,Ana Lopez,Elective procedure follow-up
3,4,3,Carlos Perez,Sepsis
4,5,3,Carlos Perez,Acute Kidney Injury
5,6,4,María Gomez,Asthma exacerbation
6,7,5,Jose Martinez,Pneumonia
7,8,6,Alex Rivera,Trauma
8,9,7,Lucía Herrera,Urinary tract infection
9,10,8,Lucía Herrera,Dehydration


In [42]:
print('=== Catálogo de labs (antes de codificar) ===')
display(pd.read_sql('SELECT * FROM d_labitems ORDER BY labitem_id;', engine))

=== Catálogo de labs (antes de codificar) ===


,labitem_id,label,unit,code_system,code,code_display
0,1,Creatinine,mg/dL,http://loinc.org,2160-0,Creatinine [Mass/volume] in Serum or Plasma
1,2,Hemoglobin,g/dL,http://loinc.org,718-7,Hemoglobin [Mass/volume] in Blood
2,3,White Blood Cells,10^9/L,http://loinc.org,6690-2,Leukocytes [#/volume] in Blood
3,4,Platelets,10^9/L,http://loinc.org,777-3,Platelets [#/volume] in Blood
4,5,Lactate,mmol/L,http://loinc.org,2524-7,Lactate [Moles/volume] in Blood


---

## Parte 2 — El servidor de terminología

Los diagnósticos usan texto libre (`'Sepsis'`, `'Hypertension'`).  
Para que otro sistema los entienda sin ambigüedad, necesitamos mapearlos a **códigos estándar**.

Vamos a construir un mini "terminology server" dentro de la misma base de datos —
son cuatro tablas nuevas, conceptualmente separadas del EHR:

```
SERVIDOR DE TERMINOLOGÍA              EHR
┌───────────────────────┐             ┌──────────────────┐
│  terminology_systems  │             │    diagnoses     │
│   ICD-10, LOINC       │             │  + code_system   │
├───────────────────────┤             │  + code          │◀──┐
│  terminology_concepts │             │  + code_display  │   │
│   A41.9 = Sepsis      │             └──────────────────┘   │ UPDATE
│   2160-0 = Creatinine │   ┌──────────────────────────┐     │
├───────────────────────┤   │ map_diagnosis_text_to_   │─────┘
│  tablas de mapeo      │──▶│ concept                  │
│  (el puente)          │   └──────────────────────────┘
└───────────────────────┘
```


### 2.1 — Crear el esquema de terminología

Este script crea las 4 tablas del terminology server y agrega las columnas
`code_system`, `code`, `code_display` a `diagnoses` y `d_labitems`.


In [43]:
ejecutar_sql('003_terminology_schema.sql')

✓ Ejecutado: 003_terminology_schema.sql


In [44]:
print('Tablas de terminología creadas:')
display(pd.read_sql('''
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_name IN (
          'terminology_systems', 'terminology_concepts',
          'map_diagnosis_text_to_concept', 'map_labitem_to_concept'
      )
    ORDER BY table_name;
''', engine))

Tablas de terminología creadas:


,table_name
0,map_diagnosis_text_to_concept
1,map_labitem_to_concept
2,terminology_concepts
3,terminology_systems


### 2.2 — Cargar los códigos ICD-10 y LOINC

Este script inserta 12 códigos ICD-10 y 5 códigos LOINC que cubren los
diagnósticos y labs del dataset, más las tablas de mapeo texto → código.


In [45]:
ejecutar_sql('004_seed_terminology.sql')

✓ Ejecutado: 004_seed_terminology.sql


In [46]:
print('=== Sistemas de terminología ===')
display(pd.read_sql('SELECT * FROM terminology_systems;', engine))

=== Sistemas de terminología ===


,system_id,system_name,system_uri,description
0,1,ICD-10,http://hl7.org/fhir/sid/icd-10,Diagnósticos (mini demo)
1,2,LOINC,http://loinc.org,Laboratorios (mini demo)


In [47]:
print('=== Conceptos ICD-10 (mini) ===')
display(pd.read_sql('''
    SELECT c.code, c.display, c.concept_type
    FROM terminology_concepts c
    JOIN terminology_systems s ON s.system_id = c.system_id
    WHERE s.system_name = 'ICD-10'
    ORDER BY c.code;
''', engine))

=== Conceptos ICD-10 (mini) ===


,code,display,concept_type
0,A41.9,"Sepsis, unspecified organism",diagnosis
1,E86.0,Dehydration,diagnosis
2,I10,Essential (primary) hypertension,diagnosis
3,I50.9,"Heart failure, unspecified",diagnosis
4,J18.9,"Pneumonia, unspecified organism",diagnosis
5,J45.901,Unspecified asthma with exacerbation,diagnosis
6,K35.80,"Acute appendicitis without perforation, absces...",diagnosis
7,N17.9,"Acute kidney failure, unspecified",diagnosis
8,N39.0,"Urinary tract infection, site not specified",diagnosis
9,R07.9,"Chest pain, unspecified",diagnosis


In [48]:
print('=== Conceptos LOINC (mini) ===')
display(pd.read_sql('''
    SELECT c.code, c.display, c.concept_type
    FROM terminology_concepts c
    JOIN terminology_systems s ON s.system_id = c.system_id
    WHERE s.system_name = 'LOINC'
    ORDER BY c.code;
''', engine))

=== Conceptos LOINC (mini) ===


,code,display,concept_type
0,2160-0,Creatinine [Mass/volume] in Serum or Plasma,lab
1,2524-7,Lactate [Moles/volume] in Blood,lab
2,6690-2,Leukocytes [#/volume] in Blood,lab
3,718-7,Hemoglobin [Mass/volume] in Blood,lab
4,777-3,Platelets [#/volume] in Blood,lab


In [49]:
print('=== Mapeos: texto local → ICD-10 ===')
display(pd.read_sql('''
    SELECT m.diagnosis_text, s.system_name, c.code, c.display
    FROM map_diagnosis_text_to_concept m
    JOIN terminology_concepts c ON c.concept_id = m.concept_id
    JOIN terminology_systems s ON s.system_id = c.system_id
    ORDER BY m.diagnosis_text;
''', engine))

=== Mapeos: texto local → ICD-10 ===


,diagnosis_text,system_name,code,display
0,Acute Kidney Injury,ICD-10,N17.9,"Acute kidney failure, unspecified"
1,Appendicitis,ICD-10,K35.80,"Acute appendicitis without perforation, absces..."
2,Asthma exacerbation,ICD-10,J45.901,Unspecified asthma with exacerbation
3,Chest pain,ICD-10,R07.9,"Chest pain, unspecified"
4,Dehydration,ICD-10,E86.0,Dehydration
5,Elective procedure follow-up,ICD-10,Z09,Follow-up exam after treatment
6,Heart failure,ICD-10,I50.9,"Heart failure, unspecified"
7,Hypertension,ICD-10,I10,Essential (primary) hypertension
8,Pneumonia,ICD-10,J18.9,"Pneumonia, unspecified organism"
9,Sepsis,ICD-10,A41.9,"Sepsis, unspecified organism"


### 2.3 — Antes / Después: aplicar los mapeos al EHR

Primero el estado **antes**: los diagnósticos tienen texto pero ningún código.


In [50]:
print('=== ANTES: diagnoses sin códigos ===')
display(pd.read_sql('''
    SELECT diagnosis_id, hadm_id, diagnosis_text, code_system, code, code_display
    FROM diagnoses
    ORDER BY diagnosis_id
    LIMIT 8;
''', engine))

=== ANTES: diagnoses sin códigos ===


,diagnosis_id,hadm_id,diagnosis_text,code_system,code,code_display
0,1,1,Hypertension,http://hl7.org/fhir/sid/icd-10,I10,Essential (primary) hypertension
1,2,1,Chest pain,http://hl7.org/fhir/sid/icd-10,R07.9,"Chest pain, unspecified"
2,3,2,Elective procedure follow-up,http://hl7.org/fhir/sid/icd-10,Z09,Follow-up exam after treatment
3,4,3,Sepsis,http://hl7.org/fhir/sid/icd-10,A41.9,"Sepsis, unspecified organism"
4,5,3,Acute Kidney Injury,http://hl7.org/fhir/sid/icd-10,N17.9,"Acute kidney failure, unspecified"
5,6,4,Asthma exacerbation,http://hl7.org/fhir/sid/icd-10,J45.901,Unspecified asthma with exacerbation
6,7,5,Pneumonia,http://hl7.org/fhir/sid/icd-10,J18.9,"Pneumonia, unspecified organism"
7,8,6,Trauma,http://hl7.org/fhir/sid/icd-10,T14.90,"Injury, unspecified, initial encounter"


In [51]:
# Aplicar mapeos: UPDATE en diagnoses y d_labitems usando las tablas de mapeo
ejecutar_sql('005_apply_mappings.sql')

✓ Ejecutado: 005_apply_mappings.sql


In [52]:
print('=== DESPUÉS: diagnoses con códigos ICD-10 ===')
display(pd.read_sql('''
    SELECT diagnosis_id, hadm_id, diagnosis_text, code_system, code, code_display
    FROM diagnoses
    ORDER BY diagnosis_id;
''', engine))

=== DESPUÉS: diagnoses con códigos ICD-10 ===


,diagnosis_id,hadm_id,diagnosis_text,code_system,code,code_display
0,1,1,Hypertension,http://hl7.org/fhir/sid/icd-10,I10,Essential (primary) hypertension
1,2,1,Chest pain,http://hl7.org/fhir/sid/icd-10,R07.9,"Chest pain, unspecified"
2,3,2,Elective procedure follow-up,http://hl7.org/fhir/sid/icd-10,Z09,Follow-up exam after treatment
3,4,3,Sepsis,http://hl7.org/fhir/sid/icd-10,A41.9,"Sepsis, unspecified organism"
4,5,3,Acute Kidney Injury,http://hl7.org/fhir/sid/icd-10,N17.9,"Acute kidney failure, unspecified"
5,6,4,Asthma exacerbation,http://hl7.org/fhir/sid/icd-10,J45.901,Unspecified asthma with exacerbation
6,7,5,Pneumonia,http://hl7.org/fhir/sid/icd-10,J18.9,"Pneumonia, unspecified organism"
7,8,6,Trauma,http://hl7.org/fhir/sid/icd-10,T14.90,"Injury, unspecified, initial encounter"
8,9,7,Urinary tract infection,http://hl7.org/fhir/sid/icd-10,N39.0,"Urinary tract infection, site not specified"
9,10,8,Dehydration,http://hl7.org/fhir/sid/icd-10,E86.0,Dehydration


In [53]:
print('=== DESPUÉS: d_labitems con códigos LOINC ===')
display(pd.read_sql('''
    SELECT labitem_id, label, unit, code_system, code, code_display
    FROM d_labitems
    ORDER BY labitem_id;
''', engine))

=== DESPUÉS: d_labitems con códigos LOINC ===


,labitem_id,label,unit,code_system,code,code_display
0,1,Creatinine,mg/dL,http://loinc.org,2160-0,Creatinine [Mass/volume] in Serum or Plasma
1,2,Hemoglobin,g/dL,http://loinc.org,718-7,Hemoglobin [Mass/volume] in Blood
2,3,White Blood Cells,10^9/L,http://loinc.org,6690-2,Leukocytes [#/volume] in Blood
3,4,Platelets,10^9/L,http://loinc.org,777-3,Platelets [#/volume] in Blood
4,5,Lactate,mmol/L,http://loinc.org,2524-7,Lactate [Moles/volume] in Blood


La base de datos está lista: datos clínicos **y** códigos estándar.  
Ahora podemos construir recursos FHIR interoperables.


---

## Parte 3 — De SQL a FHIR

FHIR define un tipo de recurso para cada concepto clínico.  
La equivalencia con nuestro modelo SQL es:

| SQL                        | FHIR Resource | Descripción                  |
| -------------------------- | ------------- | ---------------------------- |
| `patients`                 | `Patient`     | Identidad demográfica        |
| `admissions`               | `Encounter`   | Encuentro clínico / admisión |
| `diagnoses`                | `Condition`   | Diagnóstico clínico          |
| `labevents` + `d_labitems` | `Observation` | Resultado de laboratorio     |
| (todos juntos)             | `Bundle`      | Paquete de intercambio       |

Para cada recurso seguiremos este patrón:

1. **SQL → DataFrame** (la fuente de datos)
2. **Python → recurso FHIR** (la transformación)
3. **JSON FHIR** (el resultado que se intercambia)
4. **Preguntas** → consultar la especificación oficial


In [54]:
def mostrar_fhir(recurso, titulo='Recurso FHIR'):
    """Imprime el JSON de un recurso FHIR de forma legible."""
    json_str = recurso.model_dump_json(indent=2, exclude_none=True)
    sep = '=' * 60
    print(sep)
    print(f'  {titulo}')
    print(sep)
    print(json_str)
    print()
    return json_str


def fhir_dt(dt):
    """Convierte un datetime de pandas a string ISO 8601 para FHIR."""
    if dt is None or (hasattr(dt, 'isnull') and pd.isnull(dt)):
        return None
    try:
        if pd.isna(dt):
            return None
    except Exception:
        pass
    return dt.strftime('%Y-%m-%dT%H:%M:%S+00:00')


def sex_to_gender(sex):
    """Convierte el código de sexo local al vocabulario FHIR."""
    return {'M': 'male', 'F': 'female', 'O': 'other'}.get(sex, 'unknown')

In [55]:
print('=== Admisiones disponibles ===')
display(pd.read_sql('''
    SELECT
        a.hadm_id,
        p.full_name                     AS paciente,
        a.admittime::date               AS ingreso,
        a.dischtime::date               AS egreso,
        a.admission_type                AS tipo,
        a.hospital_expire_flag          AS fallecio,
        COUNT(DISTINCT d.diagnosis_id)  AS n_diagnosticos,
        COUNT(DISTINCT le.labevent_id)  AS n_labs
    FROM admissions a
    JOIN patients p ON p.subject_id = a.subject_id
    LEFT JOIN diagnoses d ON d.hadm_id = a.hadm_id
    LEFT JOIN labevents le ON le.hadm_id = a.hadm_id
    GROUP BY a.hadm_id, p.full_name, a.admittime, a.dischtime,
             a.admission_type, a.hospital_expire_flag
    ORDER BY a.hadm_id;
''', engine))

HADM_ID = 4  # Carlos Pérez — Sepsis + AKI, con muerte hospitalaria
print(f'\nTrabajando con hadm_id = {HADM_ID}')

=== Admisiones disponibles ===


,hadm_id,paciente,ingreso,egreso,tipo,fallecio,n_diagnosticos,n_labs
0,1,Ana Lopez,2101-01-10,2101-01-15,Emergency,False,2,5
1,2,Ana Lopez,2102-06-01,2102-06-05,Elective,False,1,3
2,3,Carlos Perez,2101-03-20,2101-03-28,Emergency,True,2,6
3,4,María Gomez,2101-07-11,2101-07-14,Urgent,False,2,3
4,5,Jose Martinez,2101-09-02,2101-09-10,Emergency,False,1,3
5,6,Alex Rivera,2101-12-18,2101-12-22,Emergency,False,1,3
6,7,Lucía Herrera,2101-02-05,2101-02-08,Emergency,False,1,2
7,8,Lucía Herrera,2101-11-01,2101-11-04,Urgent,False,1,3
8,9,Miguel Castillo,2101-05-14,2101-05-20,Emergency,False,1,4
9,10,Sofia Morales,2101-08-21,2101-08-24,Emergency,False,1,2



Trabajando con hadm_id = 4


---

### 3.1 — Patient (Paciente)

**Especificación oficial:** https://hl7.org/fhir/R5/patient.html

El recurso `Patient` representa la identidad demográfica de una persona.

| Campo FHIR            | Fuente SQL               | Descripción                             |
| --------------------- | ------------------------ | --------------------------------------- |
| `id`                  | `patients.subject_id`    | Identificador interno del recurso       |
| `identifier[0].value` | `patients.external_id`   | MRN del hospital                        |
| `name[0].text`        | `patients.full_name`     | Nombre completo                         |
| `gender`              | `patients.sex`           | `male` / `female` / `other` / `unknown` |
| `birthDate`           | `patients.date_of_birth` | Fecha de nacimiento                     |


In [56]:
# SQL fuente — extraemos paciente + admisión en una sola query
enc = pd.read_sql('''
    SELECT
        a.hadm_id, a.admittime, a.dischtime,
        a.admission_type, a.hospital_expire_flag,
        p.subject_id, p.external_id, p.full_name,
        p.sex, p.date_of_birth, p.date_of_death
    FROM admissions a
    JOIN patients p ON p.subject_id = a.subject_id
    WHERE a.hadm_id = %(hadm_id)s
''', engine, params={'hadm_id': HADM_ID})

print('FUENTE SQL → Patient:')
display(enc[['subject_id', 'external_id', 'full_name', 'sex', 'date_of_birth']])

FUENTE SQL → Patient:


,subject_id,external_id,full_name,sex,date_of_birth
0,3,MRN-0003,María Gomez,F,1992-11-23


In [57]:
row = enc.iloc[0]

patient = Patient.model_validate({
    'resourceType': 'Patient',
    'id': str(row['subject_id']),
    'identifier': [{'system': 'urn:uvg:mrn', 'value': row['external_id']}],
    'name': [{'text': row['full_name']}],
    'gender': sex_to_gender(row['sex']),
    'birthDate': str(row['date_of_birth']) if row['date_of_birth'] is not None else None,
})

mostrar_fhir(patient, f'Patient/{patient.id}')

  Patient/3
{
  "resourceType": "Patient",
  "id": "3",
  "identifier": [
    {
      "system": "urn:uvg:mrn",
      "value": "MRN-0003"
    }
  ],
  "name": [
    {
      "text": "María Gomez"
    }
  ],
  "gender": "female",
  "birthDate": "1992-11-23"
}



'{\n  "resourceType": "Patient",\n  "id": "3",\n  "identifier": [\n    {\n      "system": "urn:uvg:mrn",\n      "value": "MRN-0003"\n    }\n  ],\n  "name": [\n    {\n      "text": "María Gomez"\n    }\n  ],\n  "gender": "female",\n  "birthDate": "1992-11-23"\n}'

#### Ejercicio 3.1 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/patient.html** y responde:

**a)** Usamos `name[0].text` para el nombre completo. La spec también define los sub-campos `family` y `given` dentro de `HumanName`. ¿Cuándo preferirías usar `family`/`given` en lugar de `text`? ¿Qué ventaja tiene para la interoperabilidad?

**b)** ¿Qué valor de `gender` usarías para una persona cuyo sexo biológico es desconocido? ¿Y para alguien intersex?

**c)** ¿Por qué el campo `identifier.system` usa una URL (`urn:uvg:mrn`) en lugar del texto `'MRN'`?


**Tus respuestas:**

a) Usar `family` y `given` permite representar de mejor manera, estructurada y ordenada el nombre de una persona. Esto es especialmente útil en sistemas que necesitan procesar o mostrar nombres de manera específica. Al utilizar subcampos, se facilita la interoperabilidad entre diferentes sistemas de información. Por ejemplo, algunos sistemas pueden requerir el apellido (family) o el nombre de pila (given) para realizar búsquedas o clasificaciones. Esto evita confusiones y garantiza que los datos se interpreten correctamente.La especificación de subcampos ayuda a mantener un formato consistente que puede ser reconocido y utilizado por diferentes sistemas, en cuanto a la facilidad de integración los sistemas que consumen estos datos pueden mapear fácilmente los campos a sus propios modelos de datos, lo que reduce el riesgo de errores en la interpretación de nombres.

b) Para una persona cuyo sexo biológico es desconocido, se puede utilizar el valor `unknown` en el campo `gender`.Para alguien intersex, se puede utilizar el valor `other` o también `intersex`, dependiendo de la implementación y de los valores que soporte el sistema en cuestión

c) Usar una URL (como `urn:uvg:mrn`) proporciona un identificador único y globalmente reconocible, lo que evita ambigüedades. Esto es especialmente importante en sistemas de salud donde múltiples organizaciones pueden tener sus propios sistemas de identificación.Al utilizar URLs, se fomenta la estandarización de los identificadores, lo que facilita la interoperabilidad entre diferentes sistemas y organizaciones. Esto permite que los datos sean más fácilmente compartidos y comprendidos en un contexto más amplio.Las URLs pueden ser extendidas para incluir más información o para establecer relaciones con otros recursos, lo que no sería posible con un simple texto como 'MRN'.
emplos, ¡házmelo saber!


---

### 3.2 — Encounter (Encuentro Clínico)

**Especificación oficial:** https://hl7.org/fhir/R5/encounter.html

`Encounter` representa una admisión, consulta, o cualquier contacto clínico.
Equivale directamente a la tabla `admissions`.

| Campo FHIR           | Fuente SQL             | Descripción                                   |
| -------------------- | ---------------------- | --------------------------------------------- |
| `id`                 | `admissions.hadm_id`   | Identificador del encuentro                   |
| `status`             | `admissions.dischtime` | `finished` si hay egreso, `in-progress` si no |
| `subject.reference`  | `patients.subject_id`  | Referencia al Patient: `"Patient/2"`          |
| `actualPeriod.start` | `admissions.admittime` | Fecha y hora de ingreso                       |
| `actualPeriod.end`   | `admissions.dischtime` | Fecha y hora de egreso                        |


In [58]:
print('FUENTE SQL → Encounter:')
display(enc[['hadm_id', 'admittime', 'dischtime', 'admission_type', 'hospital_expire_flag']])

FUENTE SQL → Encounter:


,hadm_id,admittime,dischtime,admission_type,hospital_expire_flag
0,4,2101-07-11 13:00:00,2101-07-14 11:00:00,Urgent,False


In [59]:
encounter = Encounter.model_validate({
    'resourceType': 'Encounter',
    'id': str(row['hadm_id']),
    'status': 'finished' if row['dischtime'] is not None else 'in-progress',
    'subject': {'reference': f'Patient/{patient.id}'},
    'actualPeriod': {
        'start': fhir_dt(row['admittime']),
        'end': fhir_dt(row['dischtime'])
    },
})

mostrar_fhir(encounter, f'Encounter/{encounter.id}')

  Encounter/4
{
  "resourceType": "Encounter",
  "id": "4",
  "status": "finished",
  "subject": {
    "reference": "Patient/3"
  },
  "actualPeriod": {
    "start": "2101-07-11T13:00:00Z",
    "end": "2101-07-14T11:00:00Z"
  }
}



'{\n  "resourceType": "Encounter",\n  "id": "4",\n  "status": "finished",\n  "subject": {\n    "reference": "Patient/3"\n  },\n  "actualPeriod": {\n    "start": "2101-07-11T13:00:00Z",\n    "end": "2101-07-14T11:00:00Z"\n  }\n}'

#### Ejercicio 3.2 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/encounter.html** y responde:

**a)** ¿Cuáles son todos los valores posibles del campo `status`? Nombra al menos 4 y explica cuándo se usaría cada uno.

**b)** El Encounter tiene un campo `diagnosis` que podría referenciar Conditions. Sin embargo, nosotros ponemos la referencia al Encounter _desde_ la Condition. ¿Cuál es la diferencia entre ambos enfoques? ¿Cuándo usarías uno u otro?

**c)** ¿Qué campo de Encounter usarías para indicar el tipo de encuentro (emergencia, consulta ambulatoria, hospitalización)? ¿Qué sistema de códigos recomienda la spec?


**Tus respuestas:**

a) 

Planned: Se usa para indicar que el encuentro que ha sido planificado pero que aun no ha pasado. Se utiliza cuando se programa una cita o un procedimiento que aun no a ocurrido.

in-progress: Se utiliza cuando el evento esta en curso, como una consulta médica o algún otro procedimiento.

Onleave: Se utiliza para indicar que al paciente se le ha dado de alta de forma temporal. Esto se utiliza cuando un paciente sale del hospital por un periodo corto de tiempo.

Finished: Se utiliza cuando el encuentro ha concluido, como al terminar una consulta o tratamientoEncounter lo que hace es que contiene información sobre las condiciones diagnosticadas durante ese encuentro en especifico, mientras que condition referencia múltiples encuentros en los que se ha tratado o diagnosticado esa condición.b) . Se utilizaría Type, ya que este campo permite especificar el tipo de encuentro en varias categorías. Tales como emergencia, consulta ambulatoria, etc.c) ...


---

### 3.3 — Condition (Diagnóstico)

**Especificación oficial:** https://hl7.org/fhir/R5/condition.html

`Condition` representa un diagnóstico clínico. Gracias al Lab 2, nuestros diagnósticos
tienen **texto libre** Y **código ICD-10**.

FHIR usa `code.coding[]` para los códigos y `code.text` para el texto libre.
Vamos a ver la diferencia entre una Condition **con** código y una **sin** código.

| Campo FHIR              | Fuente SQL                 |
| ----------------------- | -------------------------- |
| `id`                    | `diagnoses.diagnosis_id`   |
| `code.coding[0].system` | `diagnoses.code_system`    |
| `code.coding[0].code`   | `diagnoses.code`           |
| `code.text`             | `diagnoses.diagnosis_text` |
| `subject.reference`     | `"Patient/{subject_id}"`   |
| `encounter.reference`   | `"Encounter/{hadm_id}"`    |


In [72]:
print('FUENTE SQL → Conditions:')
dx = pd.read_sql('''
    SELECT diagnosis_id, hadm_id, diagnosis_text,
           code_system, code, code_display
    FROM diagnoses
    WHERE hadm_id = %(hadm_id)s
    ORDER BY diagnosis_id;
''', engine, params={'hadm_id': HADM_ID})
display(dx)

FUENTE SQL → Conditions:


,diagnosis_id,hadm_id,diagnosis_text,code_system,code,code_display
0,6,4,Asthma exacerbation,http://hl7.org/fhir/sid/icd-10,J45.901,Unspecified asthma with exacerbation
1,13,4,COVID-19 pneumonia,None,None,None
2,14,4,COVID-19 pneumonia,None,None,None


In [73]:
conditions = []

for _, r in dx.iterrows():
    # Si el diagnóstico tiene código → lo incluimos en coding[]
    # Si no tiene código → coding[] queda vacío, solo queda code.text
    coding = []
    if pd.notna(r['code']) and pd.notna(r['code_system']):
        coding.append({
            'system': r['code_system'],
            'code': r['code'],
            'display': r['code_display'] if pd.notna(r['code_display']) else None
        })

    cond = Condition.model_validate({
        'resourceType': 'Condition',
        'id': str(r['diagnosis_id']),
        'clinicalStatus': {
            'coding': [{
                'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical',
                'code': 'active'
            }]
        },
        'code': {'coding': coding, 'text': r['diagnosis_text']},
        'subject': {'reference': f'Patient/{patient.id}'},
        'encounter': {'reference': f'Encounter/{encounter.id}'}
    })
    conditions.append(cond)

print(f'Se construyeron {len(conditions)} Conditions.')

Se construyeron 3 Conditions.


In [74]:
# Mostrar la primera Condition (con código ICD-10)
mostrar_fhir(conditions[0], f'Condition/{conditions[0].id} — CON código ICD-10')

  Condition/6 — CON código ICD-10
{
  "resourceType": "Condition",
  "id": "6",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active"
      }
    ]
  },
  "code": {
    "coding": [
      {
        "system": "http://hl7.org/fhir/sid/icd-10",
        "code": "J45.901",
        "display": "Unspecified asthma with exacerbation"
      }
    ],
    "text": "Asthma exacerbation"
  },
  "subject": {
    "reference": "Patient/3"
  },
  "encounter": {
    "reference": "Encounter/4"
  }
}



'{\n  "resourceType": "Condition",\n  "id": "6",\n  "clinicalStatus": {\n    "coding": [\n      {\n        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",\n        "code": "active"\n      }\n    ]\n  },\n  "code": {\n    "coding": [\n      {\n        "system": "http://hl7.org/fhir/sid/icd-10",\n        "code": "J45.901",\n        "display": "Unspecified asthma with exacerbation"\n      }\n    ],\n    "text": "Asthma exacerbation"\n  },\n  "subject": {\n    "reference": "Patient/3"\n  },\n  "encounter": {\n    "reference": "Encounter/4"\n  }\n}'

In [75]:
# Buscar alguna Condition SIN código
sin_codigo = [c for c in conditions if not c.code.coding]
if sin_codigo:
    mostrar_fhir(sin_codigo[0], f'Condition/{sin_codigo[0].id} — SIN código (solo texto)')
else:
    print('Todos los diagnósticos de esta admisión tienen código ICD-10.')
    print('En el Ejercicio B insertarás uno sin código para ver la diferencia.')

  Condition/13 — SIN código (solo texto)
{
  "resourceType": "Condition",
  "id": "13",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active"
      }
    ]
  },
  "code": {
    "coding": [],
    "text": "COVID-19 pneumonia"
  },
  "subject": {
    "reference": "Patient/3"
  },
  "encounter": {
    "reference": "Encounter/4"
  }
}



#### Ejercicio 3.3 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/condition.html** y responde:

**a)** ¿Qué valores acepta `clinicalStatus.coding[0].code` además de `active`?  
(Busca el value set `ConditionClinicalStatusCodes` en la página)

**b)** ¿Para qué sirve el campo `verificationStatus`? ¿Cuándo usarías `differential` vs `confirmed` en un contexto de urgencias?

**c)** ¿Qué campo usarías para registrar que la Sepsis comenzó 2 días antes de la admisión?  
(Busca los campos `onset*` en la spec)


**Tus respuestas:**

a) Recurrence, inactive, resolved y unknown

b) Su función es indicar el estado de verificación de la condición. Se utiliza para comunicar si la condición ha sido confirmada o si a sido descartada. Aquí se puede utilizar differential o confirmed, donde differential se utiliza cuando varias de las condiciones a consideran no se han confirmado. Y confirmed se utiliza cuando la condición ha sido confirmada atreves de pruebas, historial del paciente u otros medios.

c) Se debe utilizar el campo onsetDataTime, el cual permite especificar fecha y hora en la que comenzó la condición. 


---

### 3.4 — Observation (Resultado de Laboratorio)

**Especificación oficial:** https://hl7.org/fhir/R5/observation.html

`Observation` es el recurso más versátil de FHIR: cubre laboratorios, signos vitales,
hallazgos clínicos, y más. Cada fila de `labevents` genera una `Observation` individual.

| Campo FHIR                   | Fuente SQL                           |
| ---------------------------- | ------------------------------------ |
| `id`                         | `labevents.labevent_id`              |
| `status`                     | `'final'` (siempre, en este dataset) |
| `category[0].coding[0].code` | `'laboratory'`                       |
| `code.coding[0]`             | LOINC de `d_labitems`                |
| `code.text`                  | `d_labitems.label`                   |
| `effectiveDateTime`          | `labevents.charttime`                |
| `valueQuantity.value`        | `labevents.value_num`                |
| `valueQuantity.unit`         | `d_labitems.unit`                    |


In [64]:
print('FUENTE SQL → Observations:')
labs = pd.read_sql('''
    SELECT le.labevent_id, le.hadm_id, le.charttime, le.value_num,
           li.label, li.unit,
           li.code_system, li.code, li.code_display
    FROM labevents le
    JOIN d_labitems li ON li.labitem_id = le.labitem_id
    WHERE le.hadm_id = %(hadm_id)s
    ORDER BY le.charttime, le.labevent_id;
''', engine, params={'hadm_id': HADM_ID})
display(labs)

FUENTE SQL → Observations:


,labevent_id,hadm_id,charttime,value_num,label,unit,code_system,code,code_display
0,15,4,2101-07-12 08:00:00,11.5,Hemoglobin,g/dL,http://loinc.org,718-7,Hemoglobin [Mass/volume] in Blood
1,16,4,2101-07-12 08:00:00,9.1,White Blood Cells,10^9/L,http://loinc.org,6690-2,Leukocytes [#/volume] in Blood
2,17,4,2101-07-12 08:00:00,250.0,Platelets,10^9/L,http://loinc.org,777-3,Platelets [#/volume] in Blood


In [65]:
observations = []

for _, r in labs.iterrows():
    coding = []
    if pd.notna(r['code']) and pd.notna(r['code_system']):
        coding.append({
            'system': r['code_system'],
            'code': r['code'],
            'display': r['code_display'] if pd.notna(r['code_display']) else None
        })

    obs = Observation.model_validate({
        'resourceType': 'Observation',
        'id': str(r['labevent_id']),
        'status': 'final',
        'category': [{'coding': [{
            'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
            'code': 'laboratory'
        }]}],
        'code': {'coding': coding, 'text': r['label']},
        'subject': {'reference': f'Patient/{patient.id}'},
        'encounter': {'reference': f'Encounter/{encounter.id}'},
        'effectiveDateTime': fhir_dt(r['charttime']),
        'valueQuantity': {
            'value': float(r['value_num']),
            'unit': r['unit'],
            'system': 'http://unitsofmeasure.org',
            'code': r['unit']
        }
    })
    observations.append(obs)

print(f'Se construyeron {len(observations)} Observations.')
print('\nEjemplo — primera Observation:')
mostrar_fhir(observations[0], f'Observation/{observations[0].id}')

Se construyeron 3 Observations.

Ejemplo — primera Observation:
  Observation/15
{
  "resourceType": "Observation",
  "id": "15",
  "status": "final",
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/observation-category",
          "code": "laboratory"
        }
      ]
    }
  ],
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "718-7",
        "display": "Hemoglobin [Mass/volume] in Blood"
      }
    ],
    "text": "Hemoglobin"
  },
  "subject": {
    "reference": "Patient/3"
  },
  "encounter": {
    "reference": "Encounter/4"
  },
  "effectiveDateTime": "2101-07-12T08:00:00Z",
  "valueQuantity": {
    "value": 11.5,
    "unit": "g/dL",
    "system": "http://unitsofmeasure.org",
    "code": "g/dL"
  }
}



'{\n  "resourceType": "Observation",\n  "id": "15",\n  "status": "final",\n  "category": [\n    {\n      "coding": [\n        {\n          "system": "http://terminology.hl7.org/CodeSystem/observation-category",\n          "code": "laboratory"\n        }\n      ]\n    }\n  ],\n  "code": {\n    "coding": [\n      {\n        "system": "http://loinc.org",\n        "code": "718-7",\n        "display": "Hemoglobin [Mass/volume] in Blood"\n      }\n    ],\n    "text": "Hemoglobin"\n  },\n  "subject": {\n    "reference": "Patient/3"\n  },\n  "encounter": {\n    "reference": "Encounter/4"\n  },\n  "effectiveDateTime": "2101-07-12T08:00:00Z",\n  "valueQuantity": {\n    "value": 11.5,\n    "unit": "g/dL",\n    "system": "http://unitsofmeasure.org",\n    "code": "g/dL"\n  }\n}'

#### Ejercicio 3.4 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/observation.html** y responde:

**a)** Además de `laboratory`, ¿qué otras categorías acepta el campo `category`? ¿Cuándo usarías `vital-signs`?

**b)** Usamos `valueQuantity` para el valor numérico. ¿Qué tipo de `value[x]` usarías para un resultado de cultivo de sangre que reporta `'Staphylococcus aureus MRSA'`?

**c)** ¿Qué campo usarías para indicar que el valor de Creatinine (2.5 mg/dL) está fuera del rango de referencia normal (0.6–1.2 mg/dL)?  
(Busca `referenceRange` en la spec)


**Tus respuestas:**

a) Las categorías que acepta son: vital-signs, imaging, procedure y social-history.

Se utilizaría la categoría vital-signs cuando se quiere registrar mediciones que reflejen el estado fisiológico del paciente, tales como temperatura o la frecuencia cardiaca.

b) Se utiliza el campo de valueCodeableConcept, ya que permite representar los resultados como un concepto codificado. Este puede incluir un código de sistemas para la identificación del sujeto en especifico.

c) Se utilizaría el campo referenceRange, ya que permite especificar los rangos de referencia para el valor medido. 


---

## Parte 4 — El Bundle: el sobre del intercambio

**Especificación oficial:** https://hl7.org/fhir/R5/bundle.html

Un `Bundle` agrupa múltiples recursos FHIR para enviarlos juntos.  
Es lo que realmente se intercambia entre sistemas: no recursos sueltos, sino paquetes coherentes.

Las referencias entre recursos forman este grafo:

```
Bundle (type: collection)
│
├── Patient/2
│
├── Encounter/3
│     subject ──────────────────────────────▶ Patient/2
│
├── Condition/4
│     subject ──────────────────────────────▶ Patient/2
│     encounter ────────────────────────────▶ Encounter/3
│
├── Condition/5  (igual)
│
├── Observation/9
│     subject ──────────────────────────────▶ Patient/2
│     encounter ────────────────────────────▶ Encounter/3
│
└── Observation/10 ... (igual para cada labevent)
```


In [66]:
entries = []


def add_entry(resource):
    entries.append({'resource': json.loads(resource.model_dump_json(exclude_none=True))})


add_entry(patient)
add_entry(encounter)
for c in conditions:
    add_entry(c)
for o in observations:
    add_entry(o)

bundle = Bundle.model_validate({
    'resourceType': 'Bundle',
    'type': 'collection',
    'entry': entries
})

print('Bundle creado:')
print(f'  1  Patient')
print(f'  1  Encounter')
print(f'  {len(conditions)}  Conditions')
print(f'  {len(observations)}  Observations')
print(f'  ─────────────')
print(f'  {len(entries)}  entradas en total')

Bundle creado:
  1  Patient
  1  Encounter
  2  Conditions
  3  Observations
  ─────────────
  7  entradas en total


In [67]:
bundle_json = bundle.model_dump_json(indent=2, exclude_none=True)
print(bundle_json[:3000])
print('\n[... el JSON continúa ...]')

{
  "resourceType": "Bundle",
  "type": "collection",
  "entry": [
    {
      "resource": {
        "resourceType": "Patient",
        "id": "3",
        "identifier": [
          {
            "system": "urn:uvg:mrn",
            "value": "MRN-0003"
          }
        ],
        "name": [
          {
            "text": "María Gomez"
          }
        ],
        "gender": "female",
        "birthDate": "1992-11-23"
      }
    },
    {
      "resource": {
        "resourceType": "Encounter",
        "id": "4",
        "status": "finished",
        "subject": {
          "reference": "Patient/3"
        },
        "actualPeriod": {
          "start": "2101-07-11T13:00:00Z",
          "end": "2101-07-14T11:00:00Z"
        }
      }
    },
    {
      "resource": {
        "resourceType": "Condition",
        "id": "6",
        "clinicalStatus": {
          "coding": [
            {
              "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
              "cod

In [68]:
out_path = Path(f'bundle_hadm_{HADM_ID}.json')
out_path.write_text(bundle_json, encoding='utf-8')
print(f'Bundle guardado en: {out_path}')
print(f'Tamaño: {len(bundle_json):,} caracteres, {len(entries)} recursos')

Bundle guardado en: bundle_hadm_4.json
Tamaño: 5,017 caracteres, 7 recursos


#### Ejercicio 4.1 — Consulta la especificación

Abre **https://hl7.org/fhir/R5/bundle.html** y responde:

**a)** ¿Qué diferencia hay entre Bundle `type: collection` y `type: transaction`?  
¿Qué campo adicional tendría cada `entry` en un Bundle de tipo `transaction`?

**b)** ¿Cómo resuelve un servidor FHIR una referencia relativa como `Patient/2` dentro de un Bundle?  
¿Qué pasa si el servidor receptor no tiene ese Patient en su base de datos?


**Tus respuestas:**

a) Bundle type: collection:
Este tipo de bundle se utiliza para agrupar recursos que son considerados como una colección, sin ninguna implicación de que se deban procesar en un orden específico o que dependan unos de otros.
No hay requisitos sobre el éxito o el fracaso de las operaciones dentro del bundle; cada recurso se puede manejar de manera independiente.

Bundle type: transaction:
Este tipo de bundle se utiliza para agrupar recursos que deben ser procesados como una única unidad atómica. Esto significa que todos los recursos en el bundle deben ser procesados exitosamente o ninguno de ellos debe ser aplicado.
Si alguna operación dentro del bundle falla, todas las operaciones se revierten.
Campo adicional en cada entry en un Bundle de tipo transaction:

En un bundle de tipo transaction, cada entrada (entry) debe incluir el campo request, que especifica la acción que se debe realizar (como POST, PUT, o DELETE) y la URL del recurso.

b) e un Bundle
Cuando un servidor FHIR recibe un bundle que contiene una referencia relativa como Patient/2, el servidor resolverá esta referencia buscando el recurso correspondiente dentro del bundle mismo antes de buscar en su b base de datos:
Si el recurso Patient/2 no se encuentra en la base de datos del servidor receptor, el servidor aún puede procesar el bundle. Sin embargo, si la referencia se usa en una operación que requiere que el recurso exista (por ejemplo, al intentar vincular otro recurso a ese paciente), se pro ucirá un error.
La respuesta del servidor podría incluir un mensaje de error indicando que la referencia no es válida o que el recurso no se encontró, dependiendo del contexto y la oper dudes en pregunta


---

## Parte 5 — Ejercicios

Incluye todas las respuestas en el notebook antes de abrir tu Pull Request.


### Ejercicio A — Bundle para otra admisión

Genera el Bundle completo para una admisión **diferente** a `hadm_id = 3`.

**Instrucciones:**

1. Elige una `hadm_id` de la tabla de la Sección 3
2. Cambia el valor de `HADM_ID` en la celda correspondiente
3. Re-ejecuta todas las celdas de la Parte 3 y Parte 4
4. El Bundle se guardará automáticamente como `bundle_hadm_X.json`

**Verifica que:**

- Todas las Conditions referencian `Patient/{id}` y `Encounter/{id}` correctos
- Las Observations con código LOINC tienen `code.coding[0].code` correcto
- El archivo JSON es válido (ejecuta la celda de verificación de abajo)


In [69]:
# Verificación del Bundle generado
try:
    parsed = json.loads(out_path.read_text(encoding='utf-8'))
    tipos = {}
    for e in parsed['entry']:
        rt = e['resource']['resourceType']
        tipos[rt] = tipos.get(rt, 0) + 1
    print(f'Bundle {out_path.name} — válido')
    for k, v in tipos.items():
        print(f'  {v}  {k}')
except Exception as ex:
    print(f'Error: {ex}')

Bundle bundle_hadm_4.json — válido
  1  Patient
  1  Encounter
  2  Condition
  3  Observation


---

### Ejercicio B — Diagnóstico sin código: implicaciones

En la realidad, siempre hay diagnósticos sin mapeo. Vamos a simular ese caso.

**Instrucciones:**

1. Ejecuta la celda de abajo para insertar un diagnóstico nuevo **sin** código
2. Re-ejecuta la celda de `005_apply_mappings.sql` (es idempotente — no daña los datos existentes)
3. Re-ejecuta la Sección 3.3 (Conditions) y observa la Condition nueva
4. Responde las preguntas


In [70]:
# Insertar diagnóstico nuevo SIN cobertura en el terminology server
with engine.begin() as conn:
    conn.execute(text('''
        INSERT INTO diagnoses (hadm_id, diagnosis_text)
        VALUES (:hadm_id, 'COVID-19 pneumonia')
        ON CONFLICT DO NOTHING;
    '''), {'hadm_id': HADM_ID})
print('Diagnóstico insertado.')

Diagnóstico insertado.


In [71]:
# Re-aplicar mapeos (no afecta los ya codificados)
ejecutar_sql('005_apply_mappings.sql')

print('=== Diagnósticos actualizados ===')
display(pd.read_sql('''
    SELECT diagnosis_id, diagnosis_text, code, code_display
    FROM diagnoses
    WHERE hadm_id = %(hadm_id)s
    ORDER BY diagnosis_id;
''', engine, params={'hadm_id': HADM_ID}))

✓ Ejecutado: 005_apply_mappings.sql
=== Diagnósticos actualizados ===


,diagnosis_id,diagnosis_text,code,code_display
0,6,Asthma exacerbation,J45.901,Unspecified asthma with exacerbation
1,13,COVID-19 pneumonia,None,None
2,14,COVID-19 pneumonia,None,None


**Preguntas del Ejercicio B:**

**a)** Después de re-ejecutar la Sección 3.3, ¿cómo se ve la Condition de `'COVID-19 pneumonia'` en el JSON? ¿Tiene `code.coding[]` con datos? ¿Qué tiene en su lugar?

**b)** Si un sistema receptor busca Conditions con `code.coding[].code = 'U07.1'` (el código ICD-10 de COVID-19), ¿encontraría esta Condition? ¿Por qué?


**c)** ¿Cuál sería la solución correcta en un entorno de producción? Menciona al menos dos pasos del proceso.


**Tus respuestas:**

a) No, lo que dice es que el codigo es NONE, lo mismo para code_display, no tiene datos, no se le ha asignado un codigo

b) No la encontraria porque no iene codigo, sin codigo no puede ser recuperada con esa busqueda

c) Se debe de asignar codigos adecuados o relevantes para que se refleje la terminologia estandar. Y se deben de validas pruebas para garantizar que todo se registra y recupera correctamente. Esto incluye pruebas de interoperabilidad para verificar que los sistemas puedan acceder y utilizar los datos correctamente


---

### Ejercicio C — Bundle filtrado para registro epidemiológico

Un registro nacional solicita un Bundle con formato reducido:

- **Patient**: solo `id`, `gender` y `birthDate` (sin nombre — privacidad)
- **Encounter**: solo el período de hospitalización
- **Observations**: únicamente WBC (LOINC `6690-2`) y Lactate (LOINC `2524-7`)
- **Sin Conditions**: los diagnósticos son confidenciales para este registro

Completa el código y guarda el resultado como `bundle_registry_hadm_{HADM_ID}.json`.


In [77]:
print(row.keys())

Index(['hadm_id', 'admittime', 'dischtime', 'admission_type',
       'hospital_expire_flag', 'subject_id', 'external_id', 'full_name', 'sex',
       'date_of_birth', 'date_of_death'],
      dtype='object')


In [87]:
import json
from pathlib import Path

# ==============================
# LOINC objetivo (WBC y Lactate)
# ==============================
TARGET_LOINC = {'6690-2', '2524-7'}

# ==============================
# Patient anonimizado
# ==============================
patient_anon = {
    'resourceType': 'Patient',
    'id': str(row['subject_id']),
    'gender': row.get('sex'),
    'birthDate': str(row.get('date_of_birth'))[:10] if row.get('date_of_birth') else None
}

# ==============================
# Función para extraer códigos LOINC
# ==============================
def get_loinc_codes(obs):
    if obs.code and obs.code.coding:
        return {c.code for c in obs.code.coding if c.code}
    return set()

# ==============================
# Filtrar Observations
# ==============================
registry_obs = [
    o for o in observations
    if get_loinc_codes(o) & TARGET_LOINC
]

# ==============================
# Encounter mínimo
# ==============================
encounter_dict = {
    'resourceType': 'Encounter',
    'id': str(row['hadm_id']),
    'status': 'finished'
}

if row.get('admittime') and row.get('dischtime'):
    encounter_dict['actualPeriod'] = {
        'start': str(row['admittime']),
        'end': str(row['dischtime'])
    }

# ==============================
# Construcción del Bundle
# ==============================
entries_reg = []

# Patient
entries_reg.append({'resource': patient_anon})

# Encounter
entries_reg.append({'resource': encounter_dict})

# Observations filtradas
for o in registry_obs:
    entries_reg.append({
        'resource': json.loads(
            o.model_dump_json(exclude_none=True)
        )
    })

bundle_registry = {
    'resourceType': 'Bundle',
    'type': 'collection',
    'entry': entries_reg
}

# ==============================
# Guardar archivo
# ==============================
registry_json = json.dumps(bundle_registry, indent=2)

reg_path = Path(f'bundle_registry_hadm_{HADM_ID}.json')
reg_path.write_text(registry_json, encoding='utf-8')

print(f'Bundle de registro guardado: {reg_path}')
print([e['resource']['resourceType'] for e in entries_reg])
print(f'{len(entries_reg)} entradas ({len(registry_obs)} Observations)')

Bundle de registro guardado: bundle_registry_hadm_4.json
['Patient', 'Encounter', 'Observation']
3 entradas (1 Observations)


---

### Ejercicio D — Reflexión final

Responde en 2–4 frases cada pregunta:

**1.** ¿Qué parte del Lab 2 (terminología) fue indispensable para que los recursos FHIR tuvieran información semántica completa? ¿Qué hubiera pasado si los diagnósticos no tuvieran código ICD-10?

**2.** ¿Qué gana un hospital receptor al recibir un Bundle FHIR en lugar de un CSV o un dump SQL de las mismas tablas?

**3.** El Bundle que generaste es `type: collection`. Para enviarlo a un servidor FHIR real (como HAPI FHIR o Azure Health Data Services), ¿qué debería cambiar?  
(Busca `FHIR transaction bundle` y `fullUrl` en la spec)

**4.** ¿Qué faltaría para que este sistema sea interoperable en producción real? Menciona al menos 3 aspectos (terminología completa, seguridad, perfiles FHIR, validación, API RESTful, etc.)


**Tus respuestas:**

1. La terminologia fue la parte mas importante porque aseguraba que los recursos FHIR tuvieran informacion semantica completa, dando un marco estandarizado para codificar diagnosticos y tratamientos. Si no tuvieran este codigo se perderia la capacidad de interpretar correctamente la informacion medica, dando errores en tratamiento y comunicacion entre diferentes sistemas de salud

2. Un hospital receptor gana en eficiencia y claridad al recibir un Bundle FHIR en lugar de un CSV o un dump SQL, ya que el Bundle contiene información estructurada y semánticamente rica que es fácilmente interpretable por sistemas informáticos. Esto facilita la integración y el uso de datos clínicos, permitiendo una atención más rápida y precisa a los pacientes.

3. Para enviar el Bundle de tipo collection a un servidor FHIR real, debería cambiarse el tipo a transaction y agregar el campo fullUrl a cada recurso dentro del Bundle. Esto es necesario para que el servidor pueda identificar cómo manejar cada recurso en la transacción y asegurar que se mantenga la integridad de los datos.

4. Terminología completa: Asegurarse de que todos los códigos y descripciones estén estandarizados y actualizados.
Seguridad: Implementar medidas de seguridad adecuadas para proteger los datos sensibles de los pacientes, como autenticación y autorización.
Validación y perfiles FHIR: Utilizar perfiles FHIR específicos para garantizar que los recursos cumplan con las expectativas y requisitos de interoperabilidad establecidos en las especificaciones de FHIR.
